## Querying SIMBAD Database for a List of Stars and Using Their Object Types to Determine Binarity. 

This code uses astroquery to search SIMBAD for a given list of HIP ID's (read in as a list of numbers). 

This query will read out all of the object types allocated to each object. 

The script then searches each row for a selection of strings to suggest binarity. It will use the presence of these strings to mark the star as either a binary or a single star. 

In [13]:
# Necessary modules
from astroquery.simbad import Simbad
import pandas as pd
import numpy as np

In [3]:
# Read in the list of HIP IDs
hip_id = pd.read_csv('/Users/Jess/HGCA_survey_paper/completed_cross_match.csv')['hip_id']
print('There are ' + str(len(hip_id)) + ' stars in this list.')

2431


In [17]:
# Modifying HIP ID numbers to a format suitable for SIMBAD
full_hip_name = []
for i in range(len(hip_id)):
    new_name = 'HIP ' + str(hip_id[i])
    full_hip_name.append(new_name)

In [18]:
# Setting up a custom SIMBAD search
customSimbad = Simbad()
customSimbad.add_votable_fields('otypes')
customSimbad.get_votable_fields()

['main_id', 'coordinates', 'otypes']

In [19]:
# Querying the star list and creating a results table
result_table = customSimbad.query_objects(full_hip_name)
object_types = np.array(list(result_table['OTYPES']))

In [21]:
# Qualifying whether each star is a binary by searching object types. 
simbad_companions = []

# These object types are considerd binaries.
substring_1 = '**'
substring_2 = 'SB*'
substring_3 = 'bL*'
substring_4 = 'LXB'
substring_5 = 'Al*'
substring_6 = 'EB*'
substring_7 = '*i*'
substring_8 = 'WU*'

for i in range(len(object_types)):
    
    a = str(object_types[i]).find(substring_1)
    b = str(object_types[i]).find(substring_2)
    c = str(object_types[i]).find(substring_3)
    d = str(object_types[i]).find(substring_4)
    e = str(object_types[i]).find(substring_5)
    f = str(object_types[i]).find(substring_6)
    g = str(object_types[i]).find(substring_7)
    h = str(object_types[i]).find(substring_8)

    # If none of the substrings match in the object row, then the sum will be -8.
    sum_letters = a + b + c + d + e + f + g + h
    if sum_letters > -8:
        simbad_companions.append(1)
    else:
        simbad_companions.append(0)
        
# Saving this list to a csv file.
header = ['object_types', 'simbad_companions']
rows = list(zip(object_types, simbad_companions))
rows = pd.DataFrame(rows, columns=header)
rows.to_csv('/Users/Jess/HGCA_survey_paper/simbad_data.csv', index=False)